# 🦎 Youtube Video Downloader 🦎

## Bibliotecas

In [ ]:
import pytube

## Define o URL do vídeo

## Cria um objeto YouTube com o URL do vídeo

## Seleciona o primeiro stream de vídeo disponível

## Baixa o vídeo